# Compute Communication Overlap Trace

This notebook provides the percentage of overlap for communication and compute kernels.

In [ ]:
import pandas as pd
import plotly.offline as pyo

from IPython.display import display, HTML, Markdown
import matplotlib.pyplot as plt

import nsys_display

display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pyo.init_notebook_mode()

## Statics

this part shows the compute, communication statics

In [ ]:
# input compute, and each communication streamID.  here you need to open nsys profile to manually map the stream number to the compute, TP DP PP communication etc.
# ID should be in list, for example [7], [8,9,10]
# if no stream ID in some item, just set it to [], for example, if there is no PP, just set PP_ID = []
COMPUTE_ID =        #
TP_ID =             #
PP_ID =             #include bubble time
DP_ID =             #
Other_ID =             #


In [ ]:
stream_df = pd.read_parquet('grouped_stream.parquet')
total_duration = (stream_df["Duration"].sum()-(stream_df["Communication Overlap Duration"].sum()+stream_df["Compute Overlap Duration"].sum())/2)
if 'SubRange' in stream_df.index.names:
    subrange_total_duration = (stream_df["Duration"].groupby(level=1).sum()-(stream_df["Communication Overlap Duration"].groupby(level=1).sum()+stream_df["Compute Overlap Duration"].groupby(level=1).sum())/2)

In [ ]:
#stream_df

In [ ]:
def generate_sub_percent(df, stream_id, total_duration):
    duration = (df.loc[stream_id]["Exclude Compute Overlap Duration"]).groupby(level=1).sum()
    percent = (duration/total_duration*100).fillna(0)
    return percent

def generate_total_percent(df, stream_id, total_duration):
    duration = df.loc[stream_id]["Exclude Compute Overlap Duration"].sum()
    percent = duration/total_duration*100
    return percent

def plotpie(sizes):
    sizes_round = [round(num, 2) for num in sizes]
    labels = ['compute','tp','pp','dp','others']
    explode = (0, 0.6, 0.5, 0.4, 0.3)  # 突出第二个扇区
    colors = ['green', 'yellowgreen', 'blue', 'lightskyblue', 'gold']

    plt.pie(sizes_round, explode=explode, labels=labels, colors=colors,startangle=140)

# 创建包含数据比例的新标签
    percentages = [f'{label} - {size}%' for label, size in zip(labels, sizes_round)]

# 添加图例
    plt.legend(title="Compute Communication percent", labels=percentages, loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

    plt.axis('equal')  # 确保饼图是圆的
    plt.show()

In [ ]:
compute_percent = generate_total_percent(stream_df,COMPUTE_ID,total_duration)
tp_percent = generate_total_percent(stream_df,TP_ID,total_duration)
dp_percent = generate_total_percent(stream_df,DP_ID,total_duration)
pp_percent = generate_total_percent(stream_df,PP_ID,total_duration)
others_percent = generate_total_percent(stream_df,Other_ID,total_duration)
sizes_total = [compute_percent, tp_percent, pp_percent, dp_percent, others_percent]

if 'SubRange' in stream_df.index.names:
    subrange_compute_percent = generate_sub_percent(stream_df,COMPUTE_ID,subrange_total_duration)
    subrange_tp_percent = generate_sub_percent(stream_df,TP_ID,subrange_total_duration)
    subrange_dp_percent = generate_sub_percent(stream_df,DP_ID,subrange_total_duration)
    subrange_pp_percent = generate_sub_percent(stream_df,PP_ID,subrange_total_duration)
    subrange_others_percent = generate_sub_percent(stream_df,Other_ID,subrange_total_duration)




In [ ]:
print("the compute communication trace of total range")
plotpie(sizes_total)

In [ ]:
# if you have input subrange, input range id here, which you want to show the statics.
range_id = 1
sizes_range = [subrange_compute_percent[range_id], subrange_tp_percent[range_id], subrange_pp_percent[range_id],subrange_dp_percent[range_id], subrange_others_percent[range_id]]
print("the compute communication trace of the input subrange:",range_id)
plotpie(sizes_range)

## Per-kernel traces

The table displays overlap percentages for each kernel, corresponding to the individual rank selected from the drop-down menu.
All time values are in nanoseconds.

In [ ]:
df = pd.read_parquet('rank_trace.parquet')
nsys_display.display_table_per_rank(df)

## Grouped traces

The table presents overlap percentages for each kernel, grouped by kernel name across all ranks.

In [ ]:
grouped_df = pd.read_parquet('grouped_trace.parquet')
display(grouped_df)